___ НЕЙРОСЕТИ ___ ИИ ___  МОДЕЛИ  ___  GPU  и  CPU ___ CUDA ___

Токены  и  id

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# 1. Загружаем токенизатор и модель
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# 2. Текст для теста
text = "Привет, мир!"

# 3. Токенизация
tokens = tokenizer.tokenize(text)
token_ids = tokenizer.encode(text)

print("Токены:", tokens)
print("ID токенов:", token_ids)

# 4. Генерация текста
inputs = tokenizer.encode("Жил-был кот,", return_tensors="pt")
outputs = model.generate(inputs, max_length=30, do_sample=True, top_k=40)

print("Сгенерированный текст:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


#  Функция для показа токенов (ID и текст) (а то в 3. кракозябры)
def show_tokens(text: str):
    print("Текст:", text)
    token_ids = tokenizer.encode(text)
    tokens = tokenizer.convert_ids_to_tokens(token_ids)
    for i, (tid, tok) in enumerate(zip(token_ids, tokens)):
        # сразу нормальный вид токена через decode
        print(
            f"{i}: {tid:<6} → {tokenizer.decode([tid])!r} (сырой токен: {tok})")


# пример
show_tokens("Привет, мир!")


Токены  и  id   +  !!!  GPT (рус)  РАЗНЫЕ МОДЕЛИ (некоторые занимают много места  и  жрут ресурсы)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import textwrap

model_name = "sberbank-ai/rugpt3medium_based_on_gpt2"
""" sberbank-ai/rugpt3small_based_on_gpt2 ??
         другие модели (дольше!)  
    ai-forever/rugpt3large_based_on_gpt2 
    ai-forever/rugpt3medium_on_gpt2  ??
    ai-forever/rugpt3xl_based_on_gpt2 
    !!!   ai-forever/mGPT 
    cointegrated/rut5-base
"""

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# автоматически выбираем устройство
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Используем устройство: {device}")

text = "Привет, мир!"

# перемещаем входные данные на то же устройство
inputs = tokenizer("Когда меня ты позовёшь", return_tensors="pt").to(device)

outputs = model.generate(
    inputs["input_ids"], max_length=500, do_sample=True, top_k=40
)

novell = tokenizer.decode(outputs[0], skip_special_tokens=True)
wrapped = textwrap.fill(novell, width=60)
print(wrapped + "\n")

# ========================
# Функция для показа токенов


def show_tokens(text: str):
    print("Текст:", text)
    token_ids = tokenizer.encode(text)
    tokens = tokenizer.convert_ids_to_tokens(token_ids)
    for i, (tid, tok) in enumerate(zip(token_ids, tokens)):
        print(
            f"{i}: {tid:<6} → {tokenizer.decode([tid])!r} (сырой токен: {tok})")


# пример
show_tokens("экспроприатор', архиважнейшее подвзбзднул!")

Инфо о CUDA  GPU 

In [ ]:
import torch
print('PyTorch version:', torch.__version__)
print('CUDA available:', torch.cuda.is_available())
print('CUDA version:', torch.version.cuda)

print(torch.cuda.is_available())   # True = всё ок
print(torch.cuda.get_device_name(0))  # название карты

In [ ]:
import torch

# Проверка: видит ли CUDA
print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0))

# Создадим тензор на GPU и прогонем простую операцию
x = torch.randn(10000, 10000, device="cuda")   # создаём тензор сразу на GPU
y = torch.mm(x, x)                             # матричное умножение (нагрузка)
print("Результат:", y[0, 0].item())
print("Тензор хранится на:", y.device)


ннннн  У меня pytorch и cuda вместе работают! ^ 
А onnxruntime и cuda вместе    не работают!   v
(нужен ли? 1 https://developer.nvidia.com/cuda-downloads?target_os=Windows&target_arch=x86_64&target_version=Server2025&target_type=exe_local) 
2  https://developer.nvidia.com/cudnn 

In [ ]:
import onnxruntime as ort

# Все доступные в сборке onnxruntime провайдеры
print("Available providers:", ort.get_available_providers())

# Пытаемся создать сессию с CUDA         
session = ort.InferenceSession(
    r"C:\Users\Odins\new-git\2_1_Prog\models\model.onnx",
    providers=["CUDAExecutionProvider", "CPUExecutionProvider"]
)

print("Session providers:", session.get_providers())


Проверка версий torch ((обучатель)) и cuda (ускоритель, gpu)  + gpu

In [ ]:
import torch
print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("cuda runtime:", torch.version.cuda)
    !nvidia-smi


инфо коротко  torch и onnxruntime

In [ ]:
import torch
import onnxruntime as ort
print("torch.cuda.is_available():", torch.cuda.is_available())
print("torch.version.cuda:", torch.version.cuda)
print("CUDA device:", torch.cuda.get_device_name(
    0) if torch.cuda.is_available() else "none")
print("onnxruntime providers:", ort.get_available_providers())

onnxruntime версия 

In [ ]:
import onnxruntime as ort

print("onnxruntime ver:", ort.__version__)
print("available providers:", ort.get_available_providers())
print("default provider:", ort.get_device())


Проверка поддержки FP16 и INT8.   + Тип провайдеров ((рассчётчиков тензоров для моделей))

In [ ]:
import torch
import onnxruntime as ort

print("="*70)
print("CUDA available (torch):", torch.cuda.is_available())

if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(0)
    capability = torch.cuda.get_device_capability(0)
    print("GPU:", device_name)
    print("Compute capability:", capability)

    major, minor = capability

    # FP16
    if major >= 5:  # Pascal (5.3) и выше
        print("✅ FP16 поддерживается")
    else:
        print("❌ FP16 не поддерживается")

    # INT8
    if major >= 7:  # Turing (7.0) и выше
        print("✅ INT8 поддерживается")
    else:
        print("❌ INT8 не поддерживается (будет очень медленно или совсем не работать)")
print("="*70)
print("ONNX Runtime info:")
print("onnxruntime ver:", ort.__version__)
print("available providers:", ort.get_available_providers())

Тест  - сравнение быстроты  gpu  и  cpu в задачах ии

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import textwrap
import time

# ===== Настройки =====
prompt_text = "Жил-был кот,"
max_length = 200
top_k = 40
wrap_width = 60

# ===== Функция генерации с таймером =====
def generate_text(model, tokenizer, inputs, max_length=160, top_k=40):
    start = time.time()
    outputs = model.generate(
        inputs["input_ids"],
        max_length=max_length,
        do_sample=True,
        top_k=top_k
    )
    elapsed = time.time() - start
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return text, elapsed

 
# ===== Загрузка токенизатора и модели =====
model_name = "sberbank-ai/rugpt3medium_based_on_gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# ===== Подготовка входа =====
inputs = tokenizer(prompt_text, return_tensors="pt")

# ===== Проверка GPU =====
gpu_available = torch.cuda.is_available()
if gpu_available:
    print(f"GPU найден: {torch.cuda.get_device_name(0)}")
else:
    print("GPU не найден, используем CPU")

# ===== Генерация на GPU (если есть) =====
if gpu_available:
    device = torch.device("cuda")
    model = model.to(device)
    inputs_cuda = {k: v.to(device) for k, v in inputs.items()}

    text_gpu, time_gpu = generate_text(
        model, tokenizer, inputs_cuda, max_length, top_k)
    print("\n--- GPU ---")
    print(f"Время генерации: {time_gpu:.3f} сек")
    print(textwrap.fill(text_gpu, width=wrap_width))

# ===== Генерация на CPU =====
device = torch.device("cpu")
model = model.to(device)
inputs_cpu = {k: v.to(device) for k, v in inputs.items()}

text_cpu, time_cpu = generate_text(
    model, tokenizer, inputs_cpu, max_length, top_k)
print("\n--- CPU ---")
print(f"Время генерации: {time_cpu:.3f} сек")
print(textwrap.fill(text_cpu, width=wrap_width))



Тест   -  сравнение быстроты  gpu  и  cpu  (ПРОДВИНУТЫЙ) !!! 

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
bench_generate.py
Честный бенчмарк генерации HuggingFace (GPU vs CPU) с защитой по версиям.
Копировать/вставить и запускать.
"""

import time
import textwrap
import statistics
import inspect
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# -------------------------- Настройки --------------------------
MODEL_NAME = "sberbank-ai/rugpt3medium_based_on_gpt2"
PROMPT = "Жил-был кот,"
MAX_NEW_TOKENS = 180
TOP_K = 40
WRAP_WIDTH = 70

WARMUP_RUNS = 2
MEASURE_RUNS = 5
SEED = 42

# Режим: "sequential" (по умолчанию, безопасный) или "parallel" (требует много памяти)
RUN_MODE = "sequential"

# Для GPU: использовать fp16 (half) — да/нет
USE_FP16_ON_GPU = True

# ---------------------------------------------------------------


def print_sep():
    print("=" * 70)


def info_versions():
    import transformers
    import tokenizers
    print("transformers:", transformers.__version__)
    try:
        print("tokenizers: ", tokenizers.__version__)
    except Exception:
        print("tokenizers: <unknown>")
    print("torch:      ", torch.__version__)
    print("cuda avail: ", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("cuda ver:   ", torch.version.cuda)


def load_tokenizer(model_name=MODEL_NAME):
    print("Загрузка токенизатора...")
    return AutoTokenizer.from_pretrained(model_name)


def load_model_on_device(model_name=MODEL_NAME, device=torch.device("cpu"), use_fp16=False):
    print(
        f"Загрузка модели '{model_name}' на устройство {device} (fp16={use_fp16})...")
    model = AutoModelForCausalLM.from_pretrained(model_name)
    model.eval()
    try:
        if device.type == "cuda":
            if use_fp16:
                model.half().to(device)
            else:
                model.to(device)
        else:
            model.to(device)
    except Exception as e:
        print(
            "Предупреждение: не удалось применить fp16 / переместить модель как хотели:", e)
        model.to(device)  # попытка в любом случае
    return model


def prepare_inputs(tokenizer, prompt, device):
    t = tokenizer(prompt, return_tensors="pt")
    return {k: v.to(device) for k, v in t.items()}


def make_generator(seed: int, device: torch.device):
    try:
        gen = torch.Generator(device=device).manual_seed(seed)
    except Exception:
        gen = torch.Generator(device="cpu").manual_seed(seed)
    return gen


def generate_with_fallback(model, tokenizer, inputs, device, max_new_tokens, top_k, seed=None):
    """
    Корректный вызов model.generate с защитой от разных версий transformers:
    - если model.generate поддерживает аргумент 'generator' (см. inspect.signature) -> используем его;
    - иначе ставим глобальные сиды (torch.manual_seed / torch.cuda.manual_seed_all).
    Всегда синхронизируем CUDA до/после для корректного тайминга.
    """
    # параметры генерации
    gen_kwargs = dict(max_new_tokens=max_new_tokens,
                      do_sample=True, top_k=top_k)

    # проверяем сигнатуру
    try:
        sig = inspect.signature(model.generate)
        accepts_generator = 'generator' in sig.parameters
    except Exception:
        accepts_generator = False

    # подготовка генератора если надо
    if seed is not None and accepts_generator:
        try:
            generator = make_generator(
                seed, device if device.type == "cuda" else torch.device("cpu"))
            gen_kwargs["generator"] = generator
        except Exception:
            # fallback: не добавляем generator, будем ставить глобальные сиды
            gen_kwargs.pop("generator", None)

    # если generator не используется - ставим глобальные сиды
    if seed is not None and "generator" not in gen_kwargs:
        torch.manual_seed(seed)
        if device.type == "cuda":
            try:
                torch.cuda.manual_seed_all(seed)
            except Exception:
                pass

    # синхронизация перед измерением
    if device.type == "cuda":
        torch.cuda.synchronize()

    t0 = time.time()
    with torch.inference_mode():
        # explicit input_ids and attention_mask
        outputs = model.generate(
            inputs["input_ids"],
            attention_mask=inputs.get("attention_mask", None),
            **gen_kwargs
        )
    if device.type == "cuda":
        torch.cuda.synchronize()
    elapsed = time.time() - t0
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return text, elapsed


def run_benchmark_on_device(model, tokenizer, device, prompt, warmup=WARMUP_RUNS, runs=MEASURE_RUNS, seed=SEED):
    print(f"\nЗапуск на устройстве: {device} (warmup={warmup}, runs={runs})")
    inputs = prepare_inputs(tokenizer, prompt, device)

    # warm-up
    for i in range(warmup):
        _ = generate_with_fallback(
            model, tokenizer, inputs, device, MAX_NEW_TOKENS, TOP_K, seed=seed + i)

    times = []
    texts = []
    for i in range(runs):
        out, t = generate_with_fallback(
            model, tokenizer, inputs, device, MAX_NEW_TOKENS, TOP_K, seed=seed + i)
        times.append(t)
        texts.append(out)
        print(f"  run {i+1}/{runs}: {t:.3f}s")

    mean_t = statistics.mean(times)
    std_t = statistics.stdev(times) if len(times) > 1 else 0.0
    print(f"Результат: mean={mean_t:.3f}s std={std_t:.3f}s")
    return texts, times


def safe_unload_model(model):
    try:
        del model
    except Exception:
        pass
    try:
        torch.cuda.empty_cache()
    except Exception:
        pass


def main():
    print_sep()
    info_versions()
    print_sep()

    tokenizer = load_tokenizer(MODEL_NAME)

    gpu_available = torch.cuda.is_available()
    device_gpu = torch.device("cuda") if gpu_available else None

    # parallel mode
    if RUN_MODE == "parallel" and gpu_available:
        try:
            model_gpu = load_model_on_device(
                MODEL_NAME, device_gpu, use_fp16=USE_FP16_ON_GPU)
            model_cpu = load_model_on_device(
                MODEL_NAME, torch.device("cpu"), use_fp16=False)
            texts_gpu, times_gpu = run_benchmark_on_device(
                model_gpu, tokenizer, device_gpu, PROMPT)
            texts_cpu, times_cpu = run_benchmark_on_device(
                model_cpu, tokenizer, torch.device("cpu"), PROMPT)
            print_sep()
            print("GPU final text:")
            print(textwrap.fill(texts_gpu[-1], width=WRAP_WIDTH))
            print_sep()
            print("CPU final text:")
            print(textwrap.fill(texts_cpu[-1], width=WRAP_WIDTH))
            safe_unload_model(model_gpu)
            safe_unload_model(model_cpu)
            return
        except RuntimeError as e:
            print(
                "Ошибка при параллельной загрузке (вероятно OOM), переключаемся в sequential:", e)

    # sequential (default)
    texts_gpu, times_gpu = [], []
    if gpu_available:
        try:
            model_gpu = load_model_on_device(
                MODEL_NAME, device_gpu, use_fp16=USE_FP16_ON_GPU)
            texts_gpu, times_gpu = run_benchmark_on_device(
                model_gpu, tokenizer, device_gpu, PROMPT)
            print_sep()
            print("GPU final text:")
            print(textwrap.fill(texts_gpu[-1], width=WRAP_WIDTH))
            safe_unload_model(model_gpu)
        except Exception as e:
            print("Ошибка при запуске на GPU:", e)
            texts_gpu, times_gpu = [], []

    # CPU
    try:
        model_cpu = load_model_on_device(
            MODEL_NAME, torch.device("cpu"), use_fp16=False)
        texts_cpu, times_cpu = run_benchmark_on_device(
            model_cpu, tokenizer, torch.device("cpu"), PROMPT)
        print_sep()
        print("CPU final text:")
        print(textwrap.fill(texts_cpu[-1], width=WRAP_WIDTH))
        safe_unload_model(model_cpu)
    except Exception as e:
        print("Ошибка при запуске на CPU:", e)
        texts_cpu, times_cpu = [], []

    # Summary
    print_sep()
    if times_gpu:
        print(
            f"GPU mean: {statistics.mean(times_gpu):.3f}s (runs={len(times_gpu)})")
    if times_cpu:
        print(
            f"CPU mean: {statistics.mean(times_cpu):.3f}s (runs={len(times_cpu)})")
    print_sep()


if __name__ == "__main__":
    main()

-------------------------

НАХОДИТ ПАПКИ С МОДЕЛЯМИ  ИИ   C:\Users\Odins\.cache\huggingface\hub

In [ ]:
import os
import pathlib


def get_size(path: pathlib.Path) -> int:
    """Размер папки в байтах"""
    total = 0
    for p in path.rglob("*"):
        if p.is_file():
            total += p.stat().st_size
    return total


def find_huggingface_models():
    # Стандартные пути
    candidates = []
    home = pathlib.Path.home()
    hf_home = os.getenv("HF_HOME")

    if hf_home:
        candidates.append(pathlib.Path(hf_home))
    candidates.append(home / ".cache" / "huggingface" / "transformers")
    candidates.append(home / ".cache" / "huggingface" / "hub")

    results = []
    for base in candidates:
        if base.exists():
            for p in base.rglob("*"):
                if p.is_dir():
                    # ищем модельные папки (обычно содержат config.json или pytorch_model.bin)
                    if any((p / fname).exists() for fname in ["config.json", "pytorch_model.bin", "generation_config.json"]):
                        size_mb = get_size(p) / (1024 * 1024)
                        results.append((str(p), round(size_mb, 2)))
    return results


models = find_huggingface_models()
if not models:
    print("❌ Модели HuggingFace не найдены!")
else:
    print("✅ Найдены модели:")
    for path, size in models:
        print(f"{path} — {size} MB")

МНОГО МОДЕЛЕЙ В ОДНОМ КОДЕ (поочерёдно)
models = [
    "cointegrated/rut5-base-multitask",
    "ai-forever/ruT5-base",
    "sberbank-ai/ruT5-base"
]

for m in models:
    tok, model = load_model(m)
    # тут работа с моделью
    text = "Привет, я пошол дамой"
    inputs = tok(text, return_tensors="pt")
    out = model.generate(**inputs, max_new_tokens=30)
    print(tok.decode(out[0], skip_special_tokens=True))

    # выгружаем
    del tok, model
    gc.collect()
    torch.cuda.empty_cache()


!!!   Так можно перебирать хоть десятки моделей, не держа их все в памяти сразу

------------

пример gpt

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments

# 1. Загружаем токенизатор и модель
MODEL_NAME = "sberbank-ai/rugpt3medium_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)

# GPT2 не имеет token_type_ids → отключим
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

# 2. Датасет (пример)
texts = [
    "и вот настал рассвет",
    "вечером у костра сидели рыбаки",
    "над степью летела стая журавлей",
]

encodings = tokenizer(texts, truncation=True,
                      padding=True, return_tensors="pt")

# 3. Класс датасета


class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx])
                for key, val in self.encodings.items()}
        # ВАЖНО: labels нужны для Trainer (иначе ошибка ValueError)
        item["labels"] = item["input_ids"].clone()
        return item


dataset = TextDataset(encodings)

# 4. Аргументы тренировки
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=5,
    fp16=torch.cuda.is_available(),  # если GPU умеет FP16
)

# 5. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# 6. Запуск обучения
trainer.train()

# 7. Проверка генерации текста
prompt = "и вот настала ночь"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
output = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(output[0], skip_special_tokens=True))

пример gpt  -  генерации текста (разные стили)  -  temperature ...

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Загружаем модель один раз
MODEL_NAME = "sberbank-ai/rugpt3medium_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)

# GPT2 не имеет pad_token → ставим eos_token
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


# --- функция генерации с выбором стиля ---
def generate_text(prompt: str,
                  style: str = "neutral",
                  max_new_tokens: int = 60) -> str:
    """
    Генерация текста с творческим стилем
    style:
        'neutral'   – обычный текст
        'poetic'    – поэтический, лирический
        'descriptive' – описательный, живописный
        'dialogue'  – диалоговый, разговорный
    """

    # задаём параметры генерации по стилю
    style_params = {
        "neutral":       {"temperature": 1.0, "top_k": 50, "top_p": 0.95, "repetition_penalty": 1.2},
        "poetic":        {"temperature": 1.2, "top_k": 100, "top_p": 0.98, "repetition_penalty": 1.1},
        "descriptive":   {"temperature": 1.1, "top_k": 80, "top_p": 0.95, "repetition_penalty": 1.2},
        "dialogue":      {"temperature": 1.3, "top_k": 120, "top_p": 0.98, "repetition_penalty": 1.0},
    }

    params = style_params.get(style, style_params["neutral"])

    # можно слегка подправить промпт для стиля
    if style == "poetic":
        prompt = f"{prompt}\n—"
    elif style == "descriptive":
        prompt = f"{prompt}. "
    elif style == "dialogue":
        prompt = f'"{prompt}" — '

    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    output = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=params["temperature"],
        top_k=params["top_k"],
        top_p=params["top_p"],
        repetition_penalty=params["repetition_penalty"]
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)


# --- Примеры ---
prompt = "и вот настала ночь"

print("--- нейтральный ---")
print(generate_text(prompt, style="neutral"))

print("\n--- поэтический ---")
print(generate_text(prompt, style="poetic"))

print("\n--- описательный ---")
print(generate_text(prompt, style="descriptive"))

print("\n--- диалоговый ---")
print(generate_text(prompt, style="dialogue"))

пример gpt - (наказание за повторы)

In [ ]:
prompt = "и вот настала ночь"
max_tokens = 60 

for rp in [1.0, 1.2, 1.5]:
    print(f"\n--- repetition_penalty = {rp} ---")
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    output = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        do_sample=True,
        temperature=1.0,
        top_k=50,
        top_p=0.95,
        repetition_penalty=rp
    )
    print(tokenizer.decode(output[0], skip_special_tokens=True))

===================================================